In [1]:
try:
    from iapws import IAPWS97
except ImportError:
    !pip install iapws
    from iapws import IAPWS97

In [2]:
try:
    from pyfluids import HumidAir, InputHumidAir
except ImportError:
    !pip install pyfluids
    from pyfluids import HumidAir, InputHumidAir

Cold air enters a data center cabinet at 1 atm, 20°C, and 45 percent relative humidity at 2 m/s. A data center cabinet has a total front surface area of 1.2 m$^2$. The perforation percentage is 70\%. Heat is removed from the cabinet at a rate of 

Determine: 
- (a) the exit temperature,
- (b) the exit relative humidity of the air

In [8]:
# inputs
area_m2 = 1.2
perforation_percentage = 0.7

p1_Pa = 101325
T1_C = 20 
phi_1_percent = 45
vel1_mPerS = 2

Qin_kJperMin = 1200
Qin_kJperSec = Qin_kJperMin/60

In [5]:
# define state 1

state1 = HumidAir().with_state(
    InputHumidAir.pressure(p1_Pa),
    InputHumidAir.temperature(T1_C),
    InputHumidAir.relative_humidity(phi_1_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h1_JperKg = state1.enthalpy
h1_kJperKg = h1_JperKg/1e3; print(h1_kJperKg)
w1 = state1.humidity
v1_m3PerKg = state1.specific_volume; print(v1_m3PerKg)

# calculations
area2_m2 = perforation_percentage*area_m2

36.51374598742695
0.8334146776469603


Let's do an energy balance for this steady flow device:

$$\dot{m} h_1 + \dot{Q}_\text{in} =  \dot{m}h_2  $$

$$\implies h_2 = h_1 + \dot{Q}_\text{in}/\dot{m} $$

We need to know the mass flow rate.

In [6]:
Vdot_m3PerS = area2_m2*vel1_mPerS; print(Vdot_m3PerS)

1.68


In [7]:
mdot_kgPerS = Vdot_m3PerS/v1_m3PerKg; print(mdot_kgPerS)

2.015803231043716


In [9]:
h2_kJperKg = h1_kJperKg + (Qin_kJperSec/mdot_kgPerS); print(h2_kJperKg)

46.43534929274791


So now we have the enthalpy and pressure at state 2. However, we need one additional piece of information to be able to determine all other properties. 

Here let us assume that we are only doing "heating" cooling ($\phi_1 < \phi_2 < 100\%$). That is, there is no phase change occurring. If this is the case, then $$\omega_1 = \omega_2$$

In [10]:
w2 = w1
p2_Pa = p1_Pa
h2_JperKg = h2_kJperKg*1000

In [11]:
# define state 2

state2 = HumidAir().with_state(
    InputHumidAir.pressure(p2_Pa),
    InputHumidAir.enthalpy(h2_JperKg),
    InputHumidAir.humidity(w2),
    #InputHumidAir.temperature(T1_C),
    #InputHumidAir.relative_humidity(phi_1_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h2_JperKg = state2.enthalpy
h2_kJperKg = h2_JperKg/1e3; print(h2_kJperKg)
# w1 = state1.humidity
v2_m3PerKg = state2.specific_volume; print(v2_m3PerKg)

46.43534929274791
0.8613667786904013


In [12]:
print('(a) The exit temperature is, T2: {0:1.2f} C'.format(state2.temperature))

(a) The exit temperature is, T2: 29.80 C


In [13]:
print('(b) The exit relative humidity is: {0:1.2f} %'.format(state2.relative_humidity))

(b) The exit relative humidity is: 25.06 %
